### InceptionV3

To zaawansowana siec konwolucyjna zaprojektowana przez zespół Google w ramch modeli Inception.

Jest powszechnie używany w zadaniach z klasyfikacji obrazów i uczeniem transferowym.
Składa sie z ponad 40 warstw (konwolucyjnych, poolingowych, i w pełni połączonych)

W tradycyjnej sieci konwolucyjnej używamy tylko jednej wilekosci filtra. W Inception każdy blok równolegle przetwarza dane zróznymi filtrami:
- 1x1 do redukcji wymiarowosci danych  i uproszczenia obliczen
- 3x3 i 5x5 do znalezienia lokalnych cech obrazu na róznych skalach

##### Zastosowanie Transfer-Learning

Uzywajac pretrenowanego modelu InceptionV3
- Zaszcz

In [1]:
import pickle
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import  InceptionV3

from livelossplot import PlotLossesKeras

np.random.seed(42)


### Wczytywanie danych 

In [2]:

train_dir = r"..\input\train"
test_dir = r"..\input\test"

### Tworzenie generatora danych 
Generator bedzie skalował wartosci pikseli obrazów do zakresu [0,1] i ustalał podział danych na dane walidacyjne i treningowe

In [3]:
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(299,299),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(299,299),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")


Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.
Number of classes: 7


trian_generation i validation_generator , które będa słuzyc do ładowania obrazów w partiach z zachowaniem rozdzielczosci 75x75 pikseli
Ładowane sa w partiach po 32 obrazy

Rozmiar ten jest dostosowywany do obrazu wymaganego przez model. 

Model InceptionV3 zaprojektowano do przetwarzania obrazów w formacie RGB(3 kanały) i wymiarach 299x299. W implelentacji Keras umożliwia zmiane rozmiaru obrazu wejsciowego.

Wybieram rozmiar 75x75 poniewaz pozwala szybciej tenowac model

### Wykrywanie liczby klas

In [4]:
print(f"Train class indices: {train_generator.class_indices}")
print(f"Validation class indices: {validation_generator.class_indices}")

Train class indices: {'angry': 0, 'disgusted': 1, 'fearful': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprised': 6}
Validation class indices: {'angry': 0, 'disgusted': 1, 'fearful': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprised': 6}


### Ładowanie modelu i konfiguracja modelu bazowego
Załaduje Model bazowy InceptionV3 bez ostatniej warstwy klasyfikacyjnej (include_top= False) i dostosowany do obrazu wejsciowego

In [5]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299,3))

### Proces odmrazania warstw konwolucyjnych

- Najpierw zamrazamy wszystie warstwy modelu 
- Nastepnie okreslamy ilosc warstw które maja byc trenowane
- Potem odmrazamy warstwy które maja byc trenowane

Aby zachowac wagi wczesniejszych warstw modelu (ich wstepenie wytrenowana wiedze) pozostałem warstwy powinny byc zamrożone

In [6]:
#Zamrozenie wszystkich warstw
for layer in base_model.layers:
    layer.trainable = False

i = 30

# Odmrazanie ostatnich i warstw
for layer in base_model.layers[-i:]:
    layer.trainable = True

#Sprawdzenie statusu warstw
for i, layer in enumerate(base_model.layers):
    print(f' Layer {i} : {layer.name}, Trainable: {layer.trainable}')

 Layer 0 : input_layer, Trainable: False
 Layer 1 : conv2d, Trainable: False
 Layer 2 : batch_normalization, Trainable: False
 Layer 3 : activation, Trainable: False
 Layer 4 : conv2d_1, Trainable: False
 Layer 5 : batch_normalization_1, Trainable: False
 Layer 6 : activation_1, Trainable: False
 Layer 7 : conv2d_2, Trainable: False
 Layer 8 : batch_normalization_2, Trainable: False
 Layer 9 : activation_2, Trainable: False
 Layer 10 : max_pooling2d, Trainable: False
 Layer 11 : conv2d_3, Trainable: False
 Layer 12 : batch_normalization_3, Trainable: False
 Layer 13 : activation_3, Trainable: False
 Layer 14 : conv2d_4, Trainable: False
 Layer 15 : batch_normalization_4, Trainable: False
 Layer 16 : activation_4, Trainable: False
 Layer 17 : max_pooling2d_1, Trainable: False
 Layer 18 : conv2d_8, Trainable: False
 Layer 19 : batch_normalization_8, Trainable: False
 Layer 20 : activation_8, Trainable: False
 Layer 21 : conv2d_6, Trainable: False
 Layer 22 : conv2d_9, Trainable: False
 L

### Budowa Modelu 
Buduje model Sequential do którego dołacze model bazowy


In [7]:

model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

optimizer = Adam(0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


Model kompiluje z  optymalizatorem Adam, ustawiajac tempo ucznie 0,001 z parametrem  decay= 0.0005 oraz funkcja strat

### Konfiguracja szkolenia

In [8]:
callbacks = [
    PlotLossesKeras(),
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    
    ModelCheckpoint(filepath= r'..\models\emotion_model_InceptionV3_model_{epoch:02d}_{val_loss:.2f}.keras', 
                    monitor='val_loss', save_best_only=True, mode='min' , verbose=1),
    ReduceLROnPlateau( monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_lr=1e-6 ) 
]


- PlotLossesKeras zwraca z kazda epoką wykres strat i dokładnoscina dla zbioru treningowego i walidacyjnego w czasie trenowania
- EarlyStopping zatrzymuje trenowanie, gdy val_loss nie poprawia sie poprzez 5 epok
- ModelCheckpoint zapisuje najlepsza wersje modelu na podstawie epoki i val_loss
- ReduceLROPlateau zmniejsza tempo uczenia o wspołczynnik 0,1 gdy val_loss przestaje sie poprawiac przez 10 epok 

### Trenowanie modelu na GPU
Model bedzie trenowany przez 50 epok na GPU

In [ ]:

with tf.device('/GPU:0'):
    history = model.fit(
        train_generator,
        epochs=50,
        callbacks=callbacks,
        validation_data=validation_generator
    )


Epoch 1/50


C:\programy\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


561/718 ━━━━━━━━━━━━━━━━━━━━ 4:25:07 101s/step - accuracy: 0.2406 - loss: 4.6838

In [15]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(75,75),
    batch_size=32,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(test_generator, verbose=1)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Found 7178 images belonging to 7 classes.
225/225 ━━━━━━━━━━━━━━━━━━━━ 79s 346ms/step - accuracy: 0.4763 - loss: 1.4786
Test Loss: 1.500395655632019
Test Accuracy: 0.47199776768684387


### Podsumowanie

Transfer Learning z wykorzystaniem modelu InceptionV3 